# 인코더 LSTM과 디코더 LSTM으로 스마트 번역기 만들기

In [22]:
# 1. 패키지 수입
import numpy as np
import pandas as pd
from time import time

from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# 하이퍼 파라미터 (정확도나 속도에 영향을 미치는 값)
MY_HIDDEN = 128
MY_EPOCH  = 500


In [4]:
# 2. 번역 데이터 불러오기
raw = pd.read_csv('trans_dataset/translate.csv', header=None)
eng_kor = raw.values.tolist() # 데이터 프레임을 list로 변환
print(len(eng_kor))
print(eng_kor[:3])

110
[['cold', '감기'], ['come', '오다'], ['cook', '요리']]


In [17]:
# 3. 영어알파벳과 한글알파벳 리스트 만들기

e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
# {c:i for i, c in enumerate(e_alpha)}
k_alpha = pd.read_csv('trans_dataset/korean.csv', header=None)[0].tolist()
alpha = e_alpha + k_alpha
print('영어와 한글 알파벳 : ', alpha)
alpha_total_size = len(alpha)
print('전체 알파벳 갯수(원핫인코딩할 size) :', alpha_total_size)
print('영어 알파벳 갯수 :', len(e_alpha))
print('한글 글자수 갯수 :', len(k_alpha))

영어와 한글 알파벳 :  ['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐']
전체 알파벳 갯수(원핫인코딩할 size) : 171
영어 알파벳 갯수 : 29
한글 글자수 갯수 : 142


In [18]:
# 문자당 num를 갖는 dict 만들기
char_to_num = {c:i for i, c in enumerate(alpha)}
print(char_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [19]:
eng_kor[0]
print(char_to_num['c'], char_to_num['o'], char_to_num['l'], char_to_num['d'])
print([char_to_num[c] for c in eng_kor[0][0]])
print(char_to_num['감'], char_to_num['기'])
print([char_to_num[c] for c in eng_kor[0][1]])

5 17 14 6
[5, 17, 14, 6]
32 46
[32, 46]


In [23]:
# 원핫인코딩 방법1 : 이 코드에서는 적용 힘듦. display(pd.get_dummies([5, 3, 6]))
# 원핫인코딩 방법2
to_categorical([5, 3, 6], num_classes=10) # 0~9인덱스 원핫인코딩

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

In [27]:
# 원핫인코딩 방법1 : 이코드에서는 적용 힘듦 display(pd.get_dummies([5, 3, 6])
# 원핫인코딩 방법2
to_categorical([5, 3, 6], num_classes=10) # 0~9인덱스 원핫인코딩
# 원핫인코딩 방법3 : 단위행렬을 이용한 방법
np.eye(10)[[5,3,6]]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [49]:
# 5. 인코더 입력(영어알파벳을 숫자로 바꾼 값을 원핫인코딩), 
#    디코더 입력("S한글"을 숫자로 바꾼 값을 원핫인코딩)
#    디코더 출력("한글E"을 숫자로 바꾼 값)
def encoding(eng_kor=eng_kor):
    enc_in = [] 
    dec_in = [] 
    dec_out = []
    for data in eng_kor:
        # 인코딩 입력 데이터(영어->숫자->원핫인코딩)
        eng = [char_to_num[c] for c in data[0]]
        eng_one = np.eye(alpha_total_size)[eng]
        # eng_one = to_categorical(eng, num_classes=alpha_total_size)
        enc_in.append(eng_one)
        # 디코더 입력 데이터("S한글"->숫자->원핫인코딩)
        kor = [char_to_num[c] for c in 'S'+data[1]]
        kor_one = np.eye(alpha_total_size)[kor]
        dec_in.append(kor_one)
        # 디코더 출력 데이터 ("한글E"->숫자)
        kor = [char_to_num[c] for c in data[1]+'E']
        dec_out.append(kor)
    return enc_in, dec_in, dec_out

In [50]:
# 6. 전체 번역 데이터 처리
X_enc, X_dec, Y_dec = encoding(eng_kor)

# numpy로 전환
X_enc = np.array(X_enc)
X_dec = np.array(X_dec)
Y_dec = np.array(Y_dec).reshape(-1, 3, 1) # 차원 추가

# 최종 입출력 데이터 shape 확인
X_enc.shape, X_dec.shape, Y_dec.shape

((110, 4, 171), (110, 3, 171), (110, 3, 1))

In [51]:
# 7. 모델 구현
# 인코더 LSTM 구현
ENC_IN = Input(shape=(4, alpha_total_size))
# 출력, state_c, state_h
_, state_h, state_c = LSTM(units=MY_HIDDEN, # 128
                          return_state=True)(ENC_IN)
# 인코더와 디코더 연결
link = [state_h, state_c]

# 디코더 구현
DEC_IN = Input(shape=(3, alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN,
              return_sequences=True)(DEC_IN, initial_state=link)
# 최종 출력층
DEC_OUT = Dense(units=alpha_total_size,
               activation='softmax')(DEC_MID)

# 모델 구조 구현
model = Model(inputs=[ENC_IN, DEC_IN],
             outputs=DEC_OUT)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 4, 171)]     0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm_6 (LSTM)                  [(None, 128),        153600      ['input_7[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                            

In [52]:
# 8. 학습과정 설정 & 학습하기
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])
begin = time()
model.fit([X_enc, X_dec],
          Y_dec,
          epochs=MY_EPOCH,
          verbose=1)
end = time()
print('학습시간 :', end-begin)

Epoch 1/500
4/4 [==============================] - 4s 15ms/step - loss: 5.1231 - accuracy: 0.1758
Epoch 2/500
4/4 [==============================] - 0s 20ms/step - loss: 4.9604 - accuracy: 0.3333
Epoch 3/500
4/4 [==============================] - 0s 17ms/step - loss: 4.1111 - accuracy: 0.3333
Epoch 4/500
4/4 [==============================] - 0s 17ms/step - loss: 3.4728 - accuracy: 0.3333
Epoch 5/500
4/4 [==============================] - 0s 14ms/step - loss: 3.4171 - accuracy: 0.3333
Epoch 6/500
4/4 [==============================] - 0s 14ms/step - loss: 3.3753 - accuracy: 0.3333
Epoch 7/500
4/4 [==============================] - 0s 15ms/step - loss: 3.3400 - accuracy: 0.3333
Epoch 8/500
4/4 [==============================] - 0s 16ms/step - loss: 3.3234 - accuracy: 0.3333
Epoch 9/500
4/4 [==============================] - 0s 17ms/step - loss: 3.2858 - accuracy: 0.3333
Epoch 10/500
4/4 [==============================] - 0s 16ms/step - loss: 3.2559 - accuracy: 0.3333
Epoch 11/500
4/4 [=

4/4 [==============================] - 0s 15ms/step - loss: 1.2668 - accuracy: 0.8242
Epoch 84/500
4/4 [==============================] - 0s 18ms/step - loss: 1.2396 - accuracy: 0.8364
Epoch 85/500
4/4 [==============================] - 0s 15ms/step - loss: 1.2019 - accuracy: 0.8636
Epoch 86/500
4/4 [==============================] - 0s 16ms/step - loss: 1.1690 - accuracy: 0.8485
Epoch 87/500
4/4 [==============================] - 0s 16ms/step - loss: 1.1472 - accuracy: 0.8727
Epoch 88/500
4/4 [==============================] - 0s 16ms/step - loss: 1.1514 - accuracy: 0.8333
Epoch 89/500
4/4 [==============================] - 0s 17ms/step - loss: 1.0737 - accuracy: 0.8818
Epoch 90/500
4/4 [==============================] - 0s 16ms/step - loss: 1.0430 - accuracy: 0.9000
Epoch 91/500
4/4 [==============================] - 0s 14ms/step - loss: 1.0151 - accuracy: 0.9030
Epoch 92/500
4/4 [==============================] - 0s 15ms/step - loss: 0.9875 - accuracy: 0.9091
Epoch 93/500
4/4 [=====

4/4 [==============================] - 0s 14ms/step - loss: 0.0452 - accuracy: 1.0000
Epoch 166/500
4/4 [==============================] - 0s 15ms/step - loss: 0.0464 - accuracy: 0.9970
Epoch 167/500
4/4 [==============================] - 0s 14ms/step - loss: 0.0427 - accuracy: 0.9970
Epoch 168/500
4/4 [==============================] - 0s 15ms/step - loss: 0.0373 - accuracy: 1.0000
Epoch 169/500
4/4 [==============================] - 0s 14ms/step - loss: 0.0396 - accuracy: 1.0000
Epoch 170/500
4/4 [==============================] - 0s 14ms/step - loss: 0.0349 - accuracy: 1.0000
Epoch 171/500
4/4 [==============================] - 0s 15ms/step - loss: 0.0344 - accuracy: 1.0000
Epoch 172/500
4/4 [==============================] - 0s 16ms/step - loss: 0.0315 - accuracy: 1.0000
Epoch 173/500
4/4 [==============================] - 0s 17ms/step - loss: 0.0296 - accuracy: 1.0000
Epoch 174/500
4/4 [==============================] - 0s 16ms/step - loss: 0.0307 - accuracy: 1.0000
Epoch 175/500


4/4 [==============================] - 0s 15ms/step - loss: 4.9248e-04 - accuracy: 1.0000
Epoch 247/500
4/4 [==============================] - 0s 19ms/step - loss: 4.6740e-04 - accuracy: 1.0000
Epoch 248/500
4/4 [==============================] - 0s 15ms/step - loss: 4.4436e-04 - accuracy: 1.0000
Epoch 249/500
4/4 [==============================] - 0s 15ms/step - loss: 0.0049 - accuracy: 0.9970
Epoch 250/500
4/4 [==============================] - 0s 14ms/step - loss: 8.0368e-04 - accuracy: 1.0000
Epoch 251/500
4/4 [==============================] - 0s 14ms/step - loss: 3.5603e-04 - accuracy: 1.0000
Epoch 252/500
4/4 [==============================] - 0s 20ms/step - loss: 3.3461e-04 - accuracy: 1.0000
Epoch 253/500
4/4 [==============================] - 0s 15ms/step - loss: 3.1548e-04 - accuracy: 1.0000
Epoch 254/500
4/4 [==============================] - 0s 15ms/step - loss: 2.9889e-04 - accuracy: 1.0000
Epoch 255/500
4/4 [==============================] - 0s 15ms/step - loss: 2.8177e-

Epoch 325/500
4/4 [==============================] - 0s 14ms/step - loss: 7.5224e-06 - accuracy: 1.0000
Epoch 326/500
4/4 [==============================] - 0s 14ms/step - loss: 7.1659e-06 - accuracy: 1.0000
Epoch 327/500
4/4 [==============================] - 0s 15ms/step - loss: 6.8397e-06 - accuracy: 1.0000
Epoch 328/500
4/4 [==============================] - 0s 14ms/step - loss: 6.6114e-06 - accuracy: 1.0000
Epoch 329/500
4/4 [==============================] - 0s 14ms/step - loss: 6.4080e-06 - accuracy: 1.0000
Epoch 330/500
4/4 [==============================] - 0s 14ms/step - loss: 6.2859e-06 - accuracy: 1.0000
Epoch 331/500
4/4 [==============================] - 0s 14ms/step - loss: 5.9008e-06 - accuracy: 1.0000
Epoch 332/500
4/4 [==============================] - 0s 15ms/step - loss: 5.6386e-06 - accuracy: 1.0000
Epoch 333/500
4/4 [==============================] - 0s 14ms/step - loss: 5.5461e-06 - accuracy: 1.0000
Epoch 334/500
4/4 [==============================] - 0s 14ms/ste

4/4 [==============================] - 0s 14ms/step - loss: 1.3579e-06 - accuracy: 1.0000
Epoch 404/500
4/4 [==============================] - 0s 14ms/step - loss: 1.3377e-06 - accuracy: 1.0000
Epoch 405/500
4/4 [==============================] - 0s 14ms/step - loss: 1.3211e-06 - accuracy: 1.0000
Epoch 406/500
4/4 [==============================] - 0s 14ms/step - loss: 1.2983e-06 - accuracy: 1.0000
Epoch 407/500
4/4 [==============================] - 0s 14ms/step - loss: 1.2943e-06 - accuracy: 1.0000
Epoch 408/500
4/4 [==============================] - 0s 14ms/step - loss: 1.2748e-06 - accuracy: 1.0000
Epoch 409/500
4/4 [==============================] - 0s 14ms/step - loss: 1.2539e-06 - accuracy: 1.0000
Epoch 410/500
4/4 [==============================] - 0s 14ms/step - loss: 1.2430e-06 - accuracy: 1.0000
Epoch 411/500
4/4 [==============================] - 0s 14ms/step - loss: 1.2307e-06 - accuracy: 1.0000
Epoch 412/500
4/4 [==============================] - 0s 14ms/step - loss: 1.21

4/4 [==============================] - 0s 14ms/step - loss: 6.7841e-07 - accuracy: 1.0000
Epoch 482/500
4/4 [==============================] - 0s 14ms/step - loss: 6.7263e-07 - accuracy: 1.0000
Epoch 483/500
4/4 [==============================] - 0s 15ms/step - loss: 6.7191e-07 - accuracy: 1.0000
Epoch 484/500
4/4 [==============================] - 0s 14ms/step - loss: 6.6540e-07 - accuracy: 1.0000
Epoch 485/500
4/4 [==============================] - 0s 13ms/step - loss: 6.6504e-07 - accuracy: 1.0000
Epoch 486/500
4/4 [==============================] - 0s 14ms/step - loss: 6.5854e-07 - accuracy: 1.0000
Epoch 487/500
4/4 [==============================] - 0s 15ms/step - loss: 6.5168e-07 - accuracy: 1.0000
Epoch 488/500
4/4 [==============================] - 0s 14ms/step - loss: 6.5384e-07 - accuracy: 1.0000
Epoch 489/500
4/4 [==============================] - 0s 15ms/step - loss: 6.4987e-07 - accuracy: 1.0000
Epoch 490/500
4/4 [==============================] - 0s 15ms/step - loss: 6.40

In [53]:
# 9. 모델 사용
test = [['cold', 'PP'],
        ['down', 'PP'],
        ['desk', 'PP'],
        ['love', 'PP'],
        ['olve', 'PP'],
        ['lvoe', 'PP'],
        ['loev', 'PP'],
        ['eovl', 'PP']]
enc_in, dec_in, dec_out = encoding(test)
enc_in = np.array(enc_in)
dec_in = np.array(dec_in)
enc_in.shape, dec_in.shape

((8, 4, 171), (8, 3, 171))

In [54]:

pred = model.predict([enc_in, dec_in])
pred

1/1 [==============================] - 1s 748ms/step


array([[[3.60873983e-15, 6.43405366e-17, 4.02175733e-15, ...,
         2.41367967e-11, 6.19826621e-11, 5.67318708e-08],
        [7.23200796e-15, 1.26220726e-07, 6.83582693e-15, ...,
         1.66570191e-09, 3.83006696e-13, 2.07203407e-10],
        [3.69016149e-18, 1.00000000e+00, 3.50180708e-18, ...,
         5.52095025e-14, 9.80260311e-19, 1.80486204e-16]],

       [[3.89271745e-14, 1.41974406e-18, 4.84794835e-14, ...,
         3.48738677e-14, 6.43406942e-12, 1.02068035e-10],
        [5.71090010e-14, 4.31489049e-08, 1.18520428e-13, ...,
         3.66545705e-09, 1.37946295e-13, 6.95427917e-12],
        [3.40458607e-17, 1.00000000e+00, 4.47842036e-17, ...,
         8.97221750e-13, 7.66225839e-18, 2.57881510e-16]],

       [[4.68753014e-14, 4.02067780e-17, 2.41486739e-14, ...,
         1.06059443e-14, 6.77316397e-11, 2.02889548e-15],
        [1.46764952e-13, 3.76772356e-08, 1.23664412e-13, ...,
         7.23248267e-11, 2.10878971e-12, 1.96696657e-15],
        [2.16588800e-16, 1.00000000e